In [1]:
import psycopg2
import pandas as pd
import numpy as np
import datetime

In [2]:
con=psycopg2.connect(dbname= 'dev', host='ratecity.cvjjsart4teb.us-east-1.redshift.amazonaws.com', 
                         port= '5439', user= 'indu', password= 'Abcd@1234')
cur = con.cursor()

In [3]:
query_a=''' SELECT * from monthly_targets where month = 10 '''
cur.execute(query_a)
cur_a= cur.fetchall()
# EXTRACT(month FROM CURRENT_DATE)

In [4]:
target=pd.DataFrame(cur_a,columns=['Category','Target Sessions','Target Clicks','year','month'])

In [5]:
target=target[['Category','Target Sessions','Target Clicks']]
target

,Category,Target Sessions,Target Clicks
0,Home Loans,64347,12503
1,Savings Accounts,14589,3676
2,Other,4754,0
3,Superannuation,23375,732
4,Car Loans,43386,15021
5,Credit Cards,61886,6243
6,Term Deposits,24511,3236
7,Personal Loans,31008,14599


In [6]:
con=psycopg2.connect(dbname= 'dev', host='ratecity.cvjjsart4teb.us-east-1.redshift.amazonaws.com', 
                         port= '5439', user= 'indu', password= 'Abcd@1234')
cur = con.cursor()    

In [7]:
query_b= '''
select (case vertical
when 'superannuation' then 'Superannuation' 
when 'pension' then 'Superannuation'
when 'credit cards' then 'Credit Cards'
when 'home loans' then 'Home Loans'
when 'personal loans' then 'Personal Loans'
when 'term deposits' then 'Term Deposits'
when 'at call accounts' then 'Savings Accounts'
when  'transaction accounts'  then 'Savings Accounts'
when  'savings accounts' then 'Savings Accounts'
when 'managed funds' then 'Other'
when 'investment funds' then 'Other'
when 'margin loans' then 'Other'
when 'online Trading' then 'Other'
when 'foreign exchange' then 'Other'
when 'sale event product' then 'Other'
when 'car insurance' then 'Other'
when 'home page' then 'Other'
when 'car loans' then 'Car Loans'
when 'other' then 'Other'
when 'mortgage insurance' then 'Other'
END) as category, sum(apply_clicks) as total_clicks from user_clicks_visits
where visit_date between '2017-10-01' and '2017-10-31'
and vertical is not null
group by category
'''

In [8]:
cur.execute(query_b)
cur_b=cur.fetchall()

In [9]:
clicks=pd.DataFrame(cur_b, columns=['Category','Actual Clicks'])

In [10]:
clicks

,Category,Actual Clicks
0,Home Loans,15205
1,Car Loans,9249
2,Term Deposits,1813
3,Savings Accounts,1968
4,Superannuation,156
5,Credit Cards,11309
6,Other,87
7,Personal Loans,15655
8,None,0


In [11]:
con=psycopg2.connect(dbname= 'dev', host='ratecity.cvjjsart4teb.us-east-1.redshift.amazonaws.com', 
port= '5439', user= 'indu', password= 'Abcd@1234')
cur = con.cursor()

In [12]:
query_c='''
select (case vertical
when 'superannuation' then 'Superannuation' 
when  'pension' then 'Superannuation'
when  'credit cards' then 'Credit Cards'
when 'personal loans' then 'Personal Loans'
when  'home loans' then 'Home Loans'
when 'term deposits' then 'Term Deposits'
when 'at call accounts' then 'Savings Accounts'
when  'transaction accounts'  then 'Savings Accounts'
when  'savings accounts' then 'Savings Accounts'
when 'car loans' then 'Car Loans'
else 'Other'
END) as category, count(session_id) as total_sessions from
(select device_id, vertical, session_id from pages
where datetime between '2017-10-01' and '2017-10-31'
group by device_id, vertical, session_id)
group by category
'''

In [13]:
cur.execute(query_c)
cur_c=cur.fetchall()

In [14]:
import pandas as pd
sessions=pd.DataFrame(cur_c, columns=['Category','Actual Sessions'])

In [15]:
sessions

,Category,Actual Sessions
0,Other,28119
1,Personal Loans,36269
2,Credit Cards,63523
3,Term Deposits,18699
4,Savings Accounts,20081
5,Home Loans,114634
6,Superannuation,21906
7,Car Loans,51972


In [16]:
result_1 = pd.merge(clicks, sessions, on='Category')

In [17]:
result_1

,Category,Actual Clicks,Actual Sessions
0,Home Loans,15205,114634
1,Car Loans,9249,51972
2,Term Deposits,1813,18699
3,Savings Accounts,1968,20081
4,Superannuation,156,21906
5,Credit Cards,11309,63523
6,Other,87,28119
7,Personal Loans,15655,36269


In [18]:
target

,Category,Target Sessions,Target Clicks
0,Home Loans,64347,12503
1,Savings Accounts,14589,3676
2,Other,4754,0
3,Superannuation,23375,732
4,Car Loans,43386,15021
5,Credit Cards,61886,6243
6,Term Deposits,24511,3236
7,Personal Loans,31008,14599


In [19]:
 result = pd.merge(result_1,target,how='inner',on='Category')

In [20]:
result

,Category,Actual Clicks,Actual Sessions,Target Sessions,Target Clicks
0,Home Loans,15205,114634,64347,12503
1,Car Loans,9249,51972,43386,15021
2,Term Deposits,1813,18699,24511,3236
3,Savings Accounts,1968,20081,14589,3676
4,Superannuation,156,21906,23375,732
5,Credit Cards,11309,63523,61886,6243
6,Other,87,28119,4754,0
7,Personal Loans,15655,36269,31008,14599


In [21]:
result=result.set_index('Category')

In [22]:
result.loc['Total'] = result[['Target Clicks','Actual Clicks','Target Sessions','Actual Sessions']].sum()

In [23]:
result=result.loc[['Credit Cards','Home Loans','Personal Loans','Savings Accounts','Superannuation','Term Deposits','Car Loans','Other']]


In [24]:
result.loc['Total'] = result[['Target Clicks','Actual Clicks','Target Sessions','Actual Sessions']].sum();result

,Actual Clicks,Actual Sessions,Target Sessions,Target Clicks
Category,,,,
Credit Cards,11309,63523,61886,6243
Home Loans,15205,114634,64347,12503
Personal Loans,15655,36269,31008,14599
Savings Accounts,1968,20081,14589,3676
Superannuation,156,21906,23375,732
Term Deposits,1813,18699,24511,3236
Car Loans,9249,51972,43386,15021
Other,87,28119,4754,0
Total,55442,355203,267856,56010


In [25]:
index=['Credit Cards','Home Loans','Personal Loans','Savings Accounts','Superannuation','Term Deposits','Car Loans','Other']


In [26]:
columns=['Target CTR%']

In [27]:
new_df=pd.DataFrame(index=index,columns=columns)

In [28]:
new_df['Target CTR%']=(result['Target Clicks']/result['Target Sessions'])*100
desred_decimals = 2    


In [29]:
new_df.loc['Total']=(float(result['Target Clicks'].sum())/float(result['Target Sessions'].sum())*100)


new_df['Target CTR%'] = new_df['Target CTR%'].apply(lambda x: round(x,desred_decimals))

In [30]:
new_df

,Target CTR%
Credit Cards,10.09
Home Loans,19.43
Personal Loans,47.08
Savings Accounts,25.20
Superannuation,3.13
Term Deposits,13.20
Car Loans,34.62
Other,0.00
Total,20.91


In [31]:
my_col=['Actual CTR%']
new_df2=pd.DataFrame(index=index,columns=my_col)

In [32]:
new_df2['Actual CTR%']=(result['Actual Clicks']/result['Actual Sessions'])*100


In [33]:
new_df2.loc['Total']=(float(result['Actual Clicks'].sum())/float(result['Actual Sessions'].sum())*100)
new_df2['Actual CTR%'] = new_df2['Actual CTR%'].apply(lambda x: round(x,desred_decimals))

In [34]:
new_df2

,Actual CTR%
Credit Cards,17.80
Home Loans,13.26
Personal Loans,43.16
Savings Accounts,9.80
Superannuation,0.71
Term Deposits,9.70
Car Loans,17.80
Other,0.31
Total,15.61


In [35]:
new_df2['Actual CTR%'] = new_df2['Actual CTR%'].apply(lambda x: round(x,desred_decimals))

In [36]:
join_df=pd.merge(new_df, new_df2, left_index=True, right_index=True)

In [37]:
join_df

,Target CTR%,Actual CTR%
Credit Cards,10.09,17.80
Home Loans,19.43,13.26
Personal Loans,47.08,43.16
Savings Accounts,25.20,9.80
Superannuation,3.13,0.71
Term Deposits,13.20,9.70
Car Loans,34.62,17.80
Other,0.00,0.31
Total,20.91,15.61


In [38]:
my_result=pd.merge(result, join_df, left_index=True, right_index=True)

In [39]:
my_result=my_result[['Target Clicks','Actual Clicks','Actual Sessions','Target Sessions','Target CTR%','Actual CTR%']]
my_result

,Target Clicks,Actual Clicks,Actual Sessions,Target Sessions,Target CTR%,Actual CTR%
Category,,,,,,
Credit Cards,6243,11309,63523,61886,10.09,17.80
Home Loans,12503,15205,114634,64347,19.43,13.26
Personal Loans,14599,15655,36269,31008,47.08,43.16
Savings Accounts,3676,1968,20081,14589,25.20,9.80
Superannuation,732,156,21906,23375,3.13,0.71
Term Deposits,3236,1813,18699,24511,13.20,9.70
Car Loans,15021,9249,51972,43386,34.62,17.80
Other,0,87,28119,4754,0.00,0.31
Total,56010,55442,355203,267856,20.91,15.61


In [40]:
my_result.to_csv('daily_clicks_sessions.csv', sep=',', encoding='utf-8',index=True)

# Check current_date before sending the report